# Módulo 7 - Actividad 2:
# Del análisis al arte: procesamiento y generación de secuencias con Deep Learning

## Objetivo
Comprender el comportamiento de ambos modelos (secuencial y generativo), evaluar sus resultados y reflexionar sobre su aplicación en contextos distintos.

**Datasets utilizados:**  
`IMDb`
`MNIST`

---

### Estructura del Notebook:
1. Metodología.
2. Configuración del entorno.
3. Definicion de funciones.
4. Uso de funciones y resultados.
5. Análisis de los resultados y reflexiones finales.

---

## 1. Metodología

### Flujo de trabajo

> Para mejorar el entendimiento del código, en vez de agrupar todas las visualizaciones en una celda aparte, esta vez de decidió dejar cada visualización en el modelo que corresponden, debido a que, si bien ambas son redes neuronales, cumplen roles totalmente diferentes como lo son RNN y GAN.

1. **Carga y preprocesamiento de datos:**
    - Se cargan los dataset **IMDb** y **MNIST**, igualando las longitudes para el primer set y normalizando los datos para el segundo.

2. **Creación, entrenamiento y visualización de modelo RNN usando LSTM:**
    - Se crea un modelo LSTM, se entrena y se evalúa:
        - Se utilizó como optimizer "adam", como función de pérdida "binary_crossentropy" y como métrica "accuracy".
    - Tabla resumen de los datos de accuracy y loss para cada epoch en el modelo LSTM aplicado al dataset IMDb y tabla de accuracy y loss para el conjunto de prueba.
    - Gráficos comparativos de accuracy y loss para los datos de entrenamiento y validación.
    - Matriz de confusión de los datos de IMDb con el modelo LSTM.

3. **Creación y entrenamiento de modelo GAN:**
    - Se crea un modelo GAN con un generador y un discriminador, y luego se entrena y evalúa.
    - Datos de loss para el generador y el discriminador y accuracy cada 100 epoch en el modelo GAN con los datos de MNIST.
    - Gráfico de la evolución de accuracy y loss de GAP en función de cada epoch.
    - Grilla de imagenes guardadas generadas con GAP de los datos de IMDb cada 500 epoch.

---

# 2. Configuración del entorno

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import imdb, mnist
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, LeakyReLU, Input
from tensorflow.keras.utils import plot_model
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import os
import pandas as pd
from IPython.display import display
import matplotlib.image as mpimg
import os
import random

# Establecer seeds
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
# Crear carpeta de salida para imágenes generadas
os.makedirs("gan_images", exist_ok=True)

# 3. Definición de funciones

> **Nota:** Para mejor comprensión de las funciones y su utilidad, esta sección se divide en bloques, en donde cada uno responde a una parte diferente de la metodología de trabajo. 

---

**Bloque 1:** Carga y preprocesamiento de datos.

- **`cargar_y_preprocesar_datasets()`** 
Carga y preprocesa los datasets IMDb (con padding) y MNIST (normalización y reshape).

---

##### `Decisiones de diseño`

##### **IMDb – Clasificación de sentimientos con LSTM:**

- Razón del padding (pad_sequences):
    - Las reseñas de IMDb vienen como listas de enteros (tokens) de distinta longitud.
    - Las redes LSTM requieren que todas las secuencias tengan la misma longitud, por lo que usamos pad_sequences con maxlen=200 para truncar o rellenar con ceros.

- Elegir maxlen=200 asegura que:
    - Se conserve la mayor parte de la información importante.
    - Se mantenga una longitud manejable para el modelo (menos cómputo, menos overfitting).

##### **MNIST – Generación de dígitos con GAN:**

- Normalización (/ 255.0):
    - Los píxeles de las imágenes van de 0 a 255. Dividir por 255 los lleva al rango [0, 1], lo cual:
    - Mejora la estabilidad numérica del entrenamiento.
    - Es compatible con la salida del generador (que usa sigmoid).

- Aplanamiento (reshape(-1, 784)):
    - La GAN usa capas Dense, que requieren vectores planos. Por eso se transforma cada imagen de 28×28 a 784 dimensiones.

---

In [ ]:
def cargar_y_preprocesar_datasets():
    """
    Carga y preprocesa los datasets IMDb (para LSTM) y MNIST (para GAN).

    - Para IMDb: aplica padding a las secuencias de texto para que tengan una longitud fija.
    - Para MNIST: normaliza las imágenes y las aplana a vectores de 784 dimensiones.

    Returns:
        tuple: 
            - (x_train_imdb, y_train_imdb, x_test_imdb, y_test_imdb): Datos preprocesados del dataset IMDb.
            - x_train_mnist (ndarray): Imágenes normalizadas y aplanadas del dataset MNIST (solo datos de entrenamiento).
    """
    # IMDb: carga y padding
    num_words = 10000
    maxlen = 200
    (x_train_imdb, y_train_imdb), (x_test_imdb, y_test_imdb) = imdb.load_data(num_words=num_words)
    x_train_imdb = pad_sequences(x_train_imdb, maxlen=maxlen)
    x_test_imdb = pad_sequences(x_test_imdb, maxlen=maxlen)

    # MNIST: carga y normalización
    (x_train_mnist, _), (_, _) = mnist.load_data()
    x_train_mnist = x_train_mnist.reshape(-1, 784).astype('float32') / 255.0

    return (x_train_imdb, y_train_imdb, x_test_imdb, y_test_imdb), x_train_mnist

**Bloque 2:** Aplicación de RNN.

- **`construir_modelo_lstm()`** 
Construye y compila un modelo LSTM para clasificación binaria con embedding.

- **`entrenar_y_evaluar_lstm()`** 
Entrena el modelo LSTM y evalúa su desempeño en el conjunto test.

- **`visualizar_resultados_lstm()`** 
Muestra tablas, gráficas y matriz de confusión del entrenamiento y evaluación LSTM.

---

##### `Decisiones de diseño`

##### **Justificación del modelo LSTM:**

- Capa Embedding
    - Se utiliza una capa Embedding para transformar las secuencias de enteros (que representan palabras) en vectores densos de 128 dimensiones. Esta transformación permite que el modelo aprenda representaciones vectoriales útiles del lenguaje, donde palabras con significado similar pueden ocupar posiciones cercanas en el espacio. A diferencia de una codificación one-hot (que sería muy dispersa y de alta dimensionalidad), el embedding es eficiente y se ajusta durante el entrenamiento para captar patrones semánticos y sintácticos del texto.

- Capa LSTM
    - La capa LSTM (Long Short-Term Memory) permite al modelo procesar secuencias considerando el orden de las palabras y las dependencias a largo plazo dentro del texto. A diferencia de las redes densas, las LSTM están diseñadas específicamente para datos secuenciales, y su arquitectura con "puertas" internas permite decidir qué información conservar, olvidar o transmitir a lo largo de la secuencia. Esto es fundamental en tareas como análisis de sentimientos, donde el contexto y la posición de las palabras (por ejemplo, "no me gustó") son clave para una clasificación correcta.

- Capa Dense(1, activation='sigmoid')
    - La última capa del modelo es densa, con una sola neurona y activación sigmoid, ya que se trata de una tarea de clasificación binaria: cada reseña debe ser clasificada como positiva o negativa. La función sigmoid devuelve un valor entre 0 y 1, lo que puede interpretarse como la probabilidad de que la reseña sea positiva. Este valor probabilístico facilita tanto la interpretación como la aplicación de umbrales para la clasificación final.

- Función de pérdida binary_crossentropy
    - Se emplea la función de pérdida binary_crossentropy porque es la más adecuada para clasificación binaria. Esta función mide la divergencia entre las etiquetas verdaderas (0 o 1) y las probabilidades predichas por el modelo, penalizando más fuertemente las predicciones incorrectas que se alejan mucho del valor real. Su uso permite entrenar el modelo de forma que aprenda a asignar probabilidades más precisas a cada clase.

- Optimizador adam
    - El optimizador seleccionado es Adam, ampliamente utilizado por su eficiencia, velocidad de convergencia y estabilidad. Adam combina los beneficios del descenso por gradiente estocástico con técnicas de momentums adaptativos, ajustando la tasa de aprendizaje de cada parámetro de forma individual. Esta adaptabilidad permite que el entrenamiento sea más robusto incluso con pocos ajustes manuales.

- Métrica accuracy
    - La métrica de evaluación utilizada es la accuracy (precisión), que indica el porcentaje de predicciones correctas. Esta métrica es especialmente útil en problemas de clasificación balanceada, como este caso, ya que ofrece una medida directa e intuitiva del desempeño general del modelo en términos de aciertos sobre el total de ejemplos.

---

##### **Justificación de las visualizaciones:**

- Las gráficas de pérdida y precisión durante el entrenamiento permiten visualizar el comportamiento del modelo en las distintas épocas, diferenciando entre desempeño en entrenamiento y validación. Esto ayuda a detectar fenómenos como sobreajuste (cuando el modelo aprende muy bien el entrenamiento pero falla en validación) o infraajuste (cuando no aprende lo suficiente en ninguno de los conjuntos). Por otro lado, la matriz de confusión ofrece una visión más detallada del desempeño del modelo, mostrando cuántas veces acertó o se equivocó en cada clase, y permitiendo identificar sesgos o errores sistemáticos.

---

In [ ]:
def construir_modelo_lstm(input_length, vocab_size=10000):
    """
    Construye y compila una red neuronal LSTM para clasificación binaria de texto.

    Args:
        input_length (int): Longitud fija de las secuencias de entrada (tras padding).
        vocab_size (int, optional): Número de palabras únicas del vocabulario. Default es 10,000.

    Returns:
        keras.Model: Modelo Keras compilado listo para entrenamiento.
    """
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=128, input_length=input_length),
        LSTM(64, dropout=0.2, recurrent_dropout=0.2),
        Dense(1, activation='sigmoid')
    ])
    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    return model

def entrenar_y_evaluar_lstm(x_train, y_train, x_test, y_test, batch_size=64, epochs=3):
    """
    Entrena el modelo LSTM usando los datos de IMDb, evalúa su desempeño en el conjunto de prueba 
    y retorna los objetos necesarios para análisis posterior.

    Args:
        x_train (ndarray): Secuencias de entrenamiento (preprocesadas).
        y_train (ndarray): Etiquetas binarias de entrenamiento.
        x_test (ndarray): Secuencias de prueba.
        y_test (ndarray): Etiquetas binarias de prueba.
        batch_size (int, optional): Tamaño de lote durante entrenamiento. Default es 64.
        epochs (int, optional): Número de épocas de entrenamiento. Default es 3.

    Returns:
        tuple: 
            - history (History): Histórico de métricas del entrenamiento.
            - test_loss (float): Pérdida en el conjunto de prueba.
            - test_acc (float): Precisión en el conjunto de prueba.
            - y_test (ndarray): Etiquetas reales del test set.
            - y_pred (ndarray): Predicciones binarias generadas por el modelo.
    """
    model = construir_modelo_lstm(input_length=x_train.shape[1])

    history = model.fit(
        x_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=0.2,
        verbose=1
    )

    # Evaluación final en test set
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)

    # Predicciones
    y_pred_probs = model.predict(x_test, verbose=0)
    y_pred = (y_pred_probs > 0.5).astype('int')

    # Retornar todo lo necesario para visualizar
    return history, test_loss, test_acc, y_test, y_pred

def visualizar_resultados_lstm(history, test_loss, test_acc, y_test, y_pred):
    """
    Muestra visualizaciones y métricas clave del modelo LSTM, incluyendo:
    - Tabla resumen de entrenamiento
    - Evaluación final en test
    - Gráficas de pérdida y precisión
    - Matriz de confusión

    Args:
        history (History): Objeto de entrenamiento de Keras.
        test_loss (float): Pérdida final en el conjunto de prueba.
        test_acc (float): Precisión final en el conjunto de prueba.
        y_test (ndarray): Etiquetas verdaderas.
        y_pred (ndarray): Predicciones binarias del modelo.
    """
    import pandas as pd
    import matplotlib.pyplot as plt
    from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
    from IPython.display import display

    # === Tabla resumen de métricas por época ===
    metrics_table = pd.DataFrame({
        'Época': range(1, len(history.history['loss']) + 1),
        'Loss (train)': history.history['loss'],
        'Accuracy (train)': history.history['accuracy'],
        'Val Loss': history.history['val_loss'],
        'Val Accuracy': history.history['val_accuracy']
    })
    display(metrics_table.round(4))

    # === Display en test set ===
    eval_summary = pd.DataFrame({
        'Conjunto': ['Test'],
        'Loss': [test_loss],
        'Accuracy': [test_acc]
    })
    display(eval_summary.round(4))

    # === Gráficas de métricas ===
    plt.figure(figsize=(12, 4))

    # Pérdida
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title('Pérdida durante el entrenamiento')
    plt.xlabel('Épocas')
    plt.ylabel('Loss')
    plt.legend()

    # Precisión
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Acc')
    plt.plot(history.history['val_accuracy'], label='Val Acc')
    plt.title('Precisión durante el entrenamiento')
    plt.xlabel('Épocas')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

    # === Matriz de confusión ===
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Neg", "Pos"])
    disp.plot(cmap='Blues')
    plt.title("Matriz de Confusión - Test IMDb")
    plt.show()

**Bloque 3:** Aplicación de GAN.

- **`build_generator()`** 
Crea el modelo generador simple de la GAN que transforma ruido en imágenes 28x28.

- **`build_discriminator()`** 
Crea el modelo discriminador simple de la GAN que clasifica imágenes reales vs falsas.

- **`generate_image()`** 
Genera y guarda una imagen sintetizada por el generador en una carpeta.

- **`train_gan()`** 
Entrena la GAN completa y grafica la evolución de pérdidas y precisión.

- **`mostrar_imagenes_generadas()`** 
Muestra en una grilla las imágenes generadas guardadas durante el entrenamiento.

---

##### `Decisiones de diseño`

##### **Justificación – Entrenamiento y Evaluación de GAN:**

- Uso de LeakyReLU en el generador y discriminador
    - Se utiliza la función de activación LeakyReLU en lugar de la ReLU tradicional porque permite un pequeño gradiente incluso cuando la entrada es negativa, lo que evita el problema del “apagado de neuronas” (dead neurons) que puede ocurrir con ReLU. En GANs, donde el entrenamiento es altamente inestable y el generador parte generando ruido, es crucial mantener el flujo de gradiente incluso en regiones donde las activaciones podrían ser pequeñas o negativas. Esto estabiliza el aprendizaje tanto en el generador como en el discriminador desde las primeras iteraciones.

- ¿Cómo se genera una imagen?
    - El generador toma como entrada un vector de ruido aleatorio de 100 dimensiones, extraído de una distribución normal. Este vector no tiene estructura aparente, pero al pasar por el generador, se transforma en una imagen aplanada de 784 valores (correspondiente a 28x28 píxeles) con valores entre 0 y 1 gracias a la activación sigmoid. Esta imagen generada intenta imitar el estilo de los dígitos reales del conjunto MNIST. Al ser reconstruida y visualizada como una matriz 28x28, podemos apreciar qué tan convincentemente el generador ha aprendido la distribución visual de los números manuscritos.

- Importancia de G Loss, D Loss y Accuracy
    - G Loss (pérdida del generador): mide qué tan bien logra el generador engañar al discriminador. Cuanto más baja sea esta pérdida, más eficaz es el generador para producir imágenes que el discriminador no puede distinguir de las reales.
    - D Loss (pérdida del discriminador): evalúa la capacidad del discriminador para diferenciar imágenes reales de falsas. Una pérdida equilibrada indica que el discriminador está aprendiendo, pero no volviéndose dominante.
    - Accuracy (precisión): representa qué porcentaje de imágenes reales y falsas el discriminador clasifica correctamente. A diferencia de la pérdida, que puede ser más difícil de interpretar intuitivamente, la precisión entrega un número directo sobre su capacidad de clasificación. No se calcula una “accuracy” para el generador porque este no tiene etiquetas reales que clasificar —su objetivo es engañar, no clasificar—.

- Gráficas de las métricas G Loss, D Loss y Accuracy
    - Graficar estas dos métricas permite visualizar la dinámica del juego entre el generador y el discriminador. Un generador que mejora debería mostrar una pérdida (g_loss) decreciente o estable, mientras que un discriminador que se mantiene en 100% de precisión por mucho tiempo puede estar ganando la partida demasiado fácilmente, lo cual indica que el generador no está aprendiendo correctamente. En cambio, una evolución alternada o equilibrada refleja una mejor competencia entre ambos modelos, lo cual es deseable en una GAN. Estos gráficos permiten también detectar si alguno de los modelos colapsa (por ejemplo, si el generador deja de aprender).

- Representación de las imagenes generadas
    - Las imágenes generadas permiten evaluar visualmente si el generador está aprendiendo una distribución significativa. Como el objetivo final de la GAN es producir imágenes plausibles, observar su evolución cada cierto número de épocas permite detectar en qué momento los dígitos comienzan a tener forma coherente. Además, esto responde directamente a una de las preguntas de la tarea: ¿cuándo las imágenes comenzaron a parecerse a dígitos reales?. La grilla final con ejemplos de diferentes épocas resume de manera tangible el progreso de la GAN a lo largo del entrenamiento.

---

In [ ]:
def build_generator():
    """
    Construye el generador de la GAN.

    La red toma un vector de ruido de 100 dimensiones como entrada y produce una imagen de salida 
    de 784 píxeles (28×28 aplanada), usando capas densas y activaciones no lineales.

    Returns:
        keras.Model: Modelo generador no compilado.
    """
    model = Sequential()
    model.add(Dense(128, input_dim=100))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dense(784, activation='sigmoid'))  # salida 28x28 flatten
    return model


def build_discriminator():
    """
    Construye el discriminador de la GAN.

    La red toma una imagen aplanada de 784 píxeles como entrada y produce una probabilidad 
    de si la imagen es real o generada, usando capas densas y activación sigmoid.

    Returns:
        keras.Model: Modelo discriminador compilable.
    """
    model = Sequential()
    model.add(Dense(128, input_dim=784))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dense(1, activation='sigmoid'))  # 0 = fake, 1 = real
    return model


def generate_image(generator, epoch, output_dir="gan_images"):
    """
    Genera una imagen a partir del generador y la guarda en disco como PNG.

    Args:
        generator (keras.Model): El modelo generador entrenado.
        epoch (int): Número de época actual (para nombrar el archivo).
        output_dir (str): Carpeta donde guardar la imagen. Default es "gan_images".
    """
    noise = np.random.normal(0, 1, (1, 100))
    generated_img = generator.predict(noise, verbose=0).reshape(28, 28)

    plt.imshow(generated_img, cmap='gray')
    plt.axis('off')
    plt.title(f'Generado en epoch {epoch}')
    plt.savefig(f"{output_dir}/digit_epoch_{epoch}.png")
    plt.close()


def train_gan(x_train, epochs=3000, batch_size=128, save_interval=500):
    """
    Entrena una GAN básica para generar dígitos manuscritos.

    Se entrena el discriminador para distinguir imágenes reales de generadas, y el generador
    para engañar al discriminador. Se guarda una imagen generada cada cierto intervalo y
    se grafica la evolución de la pérdida del generador y la precisión del discriminador.

    Args:
        x_train (ndarray): Imágenes reales (aplanadas y normalizadas).
        epochs (int): Número de iteraciones de entrenamiento. Default es 3000.
        batch_size (int): Tamaño de lote total (mitad real, mitad generado).
        save_interval (int): Intervalo de épocas para guardar imágenes generadas.
    """
    g_losses = []
    d_accuracies = []

    half_batch = batch_size // 2

    generator = build_generator()
    discriminator = build_discriminator()

    # Compilar discriminador
    discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Construir GAN completa
    discriminator.trainable = False
    gan_input = Input(shape=(100,))
    gan_output = discriminator(generator(gan_input))
    gan = Model(gan_input, gan_output)
    gan.compile(loss='binary_crossentropy', optimizer='adam')

    for epoch in range(1, epochs + 1):
        # === Entrenamiento del discriminador ===
        idx = np.random.randint(0, x_train.shape[0], half_batch)
        real_imgs = x_train[idx]

        noise = np.random.normal(0, 1, (half_batch, 100))
        fake_imgs = generator.predict(noise, verbose=0)

        d_loss_real = discriminator.train_on_batch(real_imgs, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(fake_imgs, np.zeros((half_batch, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # === Entrenamiento del generador (vía GAN) ===
        noise = np.random.normal(0, 1, (batch_size, 100))
        g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

        # === Mostrar progreso ===
        if epoch % 100 == 0:
            print(f"Epoch {epoch} — D loss: {d_loss[0]:.4f}, acc: {d_loss[1]*100:.2f}% — G loss: {g_loss:.4f}")

        # === Guardar imagen generada ===
        if epoch % save_interval == 0:
            generate_image(generator, epoch)

        g_losses.append(g_loss)
        d_accuracies.append(d_loss[1])  # d_loss = [loss, acc]

    # === Gráfico de evolución G Loss y D Accuracy ===
    plt.figure(figsize=(10, 5))
    plt.plot(g_losses, label='Generador - Pérdida', linewidth=2)
    plt.plot(d_accuracies, label='Discriminador - Accuracy', linewidth=2)
    plt.xlabel('Épocas')
    plt.title('Evolución del entrenamiento GAN')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def mostrar_imagenes_generadas(carpeta='gan_images', intervalo=500, max_epoch=5000):
    """
    Muestra en una grilla las imágenes generadas por la GAN en diferentes épocas.

    Args:
        carpeta (str): Ruta a la carpeta donde están las imágenes PNG.
        intervalo (int): Frecuencia con la que se guardaron imágenes (ej: cada 500 epochs).
        max_epoch (int): Última época considerada.
    """
    epochs = list(range(intervalo, max_epoch + 1, intervalo))
    total = len(epochs)
    cols = 5
    rows = (total + cols - 1) // cols

    fig, axes = plt.subplots(rows, cols, figsize=(cols * 2, rows * 2))
    axes = axes.flatten()

    for i, epoch in enumerate(epochs):
        ruta = os.path.join(carpeta, f'digit_epoch_{epoch}.png')
        if os.path.exists(ruta):
            img = mpimg.imread(ruta)
            axes[i].imshow(img, cmap='gray')
            axes[i].axis('off')
            axes[i].set_title(f"Epoch {epoch}")
        else:
            axes[i].axis('off')
            axes[i].set_title(f"Epoch {epoch}\n(no encontrada)")

    # Quitar ejes vacíos
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')

    plt.suptitle("Evolución de imágenes generadas por la GAN", fontsize=16)
    plt.tight_layout()
    plt.subplots_adjust(top=0.9)  # deja espacio al título
    plt.show()

**Bloque 4:** Función de ejecución main.

- **`main()`** 
Ejecuta todo el flujo: carga datos, entrena LSTM y GAN, y visualiza resultados.

---

In [ ]:
def main():
    """
    Función principal que ejecuta el flujo completo de la tarea.

    1. Carga y preprocesa los datasets IMDb (para clasificación de sentimientos) y MNIST (para generación de dígitos).
    2. Entrena una red LSTM con el dataset IMDb y visualiza sus resultados, incluyendo métricas, gráficas y matriz de confusión.
    3. Entrena una red GAN simple con MNIST durante 3000 épocas, guarda imágenes generadas en intervalos regulares,
       y grafica la evolución de la pérdida del generador y la precisión del discriminador.
    4. Muestra en una grilla las imágenes generadas por la GAN a lo largo del entrenamiento para observar el progreso visual.

    Esta función organiza todo el flujo del proyecto, facilitando la ejecución estructurada y replicable.
    """
    # === 1. Cargar datasets ===
    (imdb_train_x, imdb_train_y, imdb_test_x, imdb_test_y), mnist_train_x = cargar_y_preprocesar_datasets()

    # === 2. Parte A: LSTM con IMDb ===
    history, test_loss, test_acc, y_test, y_pred = entrenar_y_evaluar_lstm(
        imdb_train_x, imdb_train_y, imdb_test_x, imdb_test_y
    )
    visualizar_resultados_lstm(history, test_loss, test_acc, y_test, y_pred)

    # === 3. Parte B: GAN con MNIST ===
    train_gan(mnist_train_x, epochs=5000, batch_size=128, save_interval=500)
    mostrar_imagenes_generadas()

# 4. Visualización de resultados

Se muestran los resultados obtenidos a partir de la ejecución de la funcion **main()**.

---

In [ ]:
if __name__ == "__main__":
    main()

# 5. Análisis de los resultados y reflexiones finales

---

## Reflexión con base en los resultados obtenidos con RNN

- Los resultados muestran que la red LSTM logra una buena precisión en entrenamiento y validación (alrededor del 85-87% en validación y test), lo que indica que efectivamente está aprendiendo a clasificar el sentimiento de las reseñas. Sin embargo, se observa que la pérdida de validación no disminuye consistentemente y el accuracy no mejora después de cierto punto, sugiriendo que la red tiene ciertas dificultades para generalizar más allá de los patrones más evidentes.

- Esto puede relacionarse con la dificultad que tiene la red para “recordar” secuencias completas o información a largo plazo en las reseñas. Como las reseñas son textos relativamente largos, la LSTM podría estar olvidando detalles importantes que aparecen al principio o en secciones intermedias del texto, afectando su capacidad de clasificación fina.

- La matriz de confusión apoya esta idea: hay un número considerable de falsos negativos y falsos positivos, lo que indica que la red a veces confunde la polaridad del sentimiento, probablemente porque no captura el contexto completo o matices en la secuencia.

- En resumen, aunque la LSTM aprende patrones relevantes, la complejidad y longitud de las secuencias representan un reto para su memoria, limitando su capacidad para “recordar” toda la información necesaria para una clasificación perfecta.

---

## Reflexión con base en los resultados obtenidos con GAN

- Durante el entrenamiento de la GAN, observamos que la precisión del discriminador se mantiene muy alta, cercana a 1 (100%), prácticamente durante todo el proceso. Esto indica que el discriminador logra distinguir con gran éxito entre imágenes reales y falsas generadas, lo que es esperable al principio cuando el generador todavía produce imágenes poco realistas.

- Sin embargo, la pérdida del discriminador varía bastante a lo largo del entrenamiento. Esto puede ser interpretado como que, aunque el discriminador es muy bueno clasificando, su confianza o certeza (reflejada en la pérdida) fluctúa conforme el generador mejora y genera imágenes más convincentes. En momentos donde la pérdida baja, el discriminador se siente seguro; cuando sube, está enfrentando imágenes más difíciles de clasificar.

- En cuanto al generador, la pérdida tiende a incrementarse de forma general, lo que se puede interpretar como un proceso de mejora en su capacidad para engañar al discriminador. Una pérdida alta del generador significa que está produciendo imágenes que el discriminador encuentra cada vez más difíciles de diferenciar de las reales.

- En las imágenes generadas, vemos que recién alrededor de la época 4000 aparecen formas que se parecen a dígitos reales, aunque con cierta variabilidad entre ejecuciones.

- En resumen, la alta precisión del discriminador junto con la variabilidad de su pérdida y el aumento de la pérdida del generador reflejan la dinámica típica del entrenamiento GAN, donde ambos modelos están en una especie de “competencia” que hace que la calidad de las imágenes mejoren gradualmente hasta volverse reconocibles.

---